In [1]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
from scipy.interpolate import griddata

import tensorflow as tf
from tensorflow.keras import layers, models

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-12-14 19:23:48.008150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# get all the data & process it nicely

In [2]:
client = pd.read_csv("../../data/client.csv")
ePrices = pd.read_csv("../../data/electricity_prices.csv")
gasPrices = pd.read_csv("../../data/gas_prices.csv")

weatherPredInt = pd.read_csv("interpolPredWeather.csv")
weatherHistInt = pd.read_csv("interpolHistOverlapWeather.csv")

train = pd.read_csv("../../data/train.csv")

### create features historical weather

In [3]:
# get the diff features
weatherHistInt['cloudcover_total_histint'] = weatherHistInt['cloudcover_total_histint'] / 100
weatherHistInt['surface_solar_radiation_downwards_histint'] = weatherHistInt['diffuse_radiationint'] + weatherHistInt['direct_solar_radiation_histint'] #target['shortwave_radiation']#
weatherHistInt['snowfall_predint'] = 1000/200 * 100 * weatherHistInt['snowfall_predint']
weatherHistInt['10_metre_u_wind_component_histint'] = np.cos(weatherHistInt['winddirection_10mint']) * weatherHistInt['windspeed_10mint']
weatherHistInt['10_metre_v_wind_component_histint'] = np.sin(weatherHistInt['winddirection_10mint']) * weatherHistInt['windspeed_10mint']

weatherHistInt['diff_temperature']                       = abs(weatherHistInt['temperature_predint'] - weatherHistInt['temperature_histint'])#/weatherHistInt['temperature_hist'] #no normalization, otherwise numerical problems when temp close to 0
weatherHistInt['diff_dewpoint']                          = abs(weatherHistInt['dewpoint_predint'] - weatherHistInt['dewpoint_histint'])#/weatherHistInt['dewpoint_hist']
weatherHistInt['diff_cloudcover_total']                  = abs(weatherHistInt['cloudcover_total_predint'] - weatherHistInt['cloudcover_total_histint']) # Cloudcover is already in %!!

#max_dirRad_per_day = weatherHistInt.groupby('Date')['direct_solar_radiation_hist'].transform('max')
weatherHistInt['diff_direct_solar_radiation']            = abs(weatherHistInt['direct_solar_radiation_predint'] - weatherHistInt['direct_solar_radiation_histint'])#/max_dirRad_per_day

# normalize over the maximum day value to get rid of seasonal effects
#max_solRad_per_day = weatherHistInt.groupby('Date')['surface_solar_radiation_downwards_hist'].transform('max')
weatherHistInt['diff_surface_solar_radiation_downwards'] = abs(weatherHistInt['surface_solar_radiation_downwardsint'] - weatherHistInt['surface_solar_radiation_downwards_histint'])#/max_solRad_per_day
weatherHistInt['diff_snowfall']                          = abs(weatherHistInt['snowfall_predint'] - weatherHistInt['snowfall_histint'])#/weatherHistInt['snowfall_hist']
weatherHistInt['diff_10_metre_u_wind_component']         = abs(weatherHistInt['10_metre_u_wind_componentint'] - weatherHistInt['10_metre_u_wind_component_histint'])#/(abs(weatherHistInt['10_metre_u_wind_component_hist']))
weatherHistInt['diff_10_metre_v_wind_component']         = abs(weatherHistInt['10_metre_u_wind_componentint'] - weatherHistInt['10_metre_v_wind_component_histint'])#/abs(weatherHistInt['10_metre_v_wind_component_hist'])


In [4]:
histWfeat = [ #'latitude', 'longitude', 
        'County', 'forecast_datetime',
       'hours_ahead', 'data_block_id',# 'origin_datetime',
       #'temperature_predint', 'dewpoint_predint', 
       #'cloudcover_high_predint','cloudcover_low_predint', 'cloudcover_mid_predint',
       #'cloudcover_total_predint', '10_metre_u_wind_componentint',
       #'10_metre_v_wind_componentint', 'direct_solar_radiation_predint',
       #'surface_solar_radiation_downwardsint', 'snowfall_predint',
       #'total_precipitationint', 'temperature_histint', 'dewpoint_histint',
       #'rainint', 'snowfall_histint', 'surface_pressureint',
       #'cloudcover_total_histint', 'cloudcover_low_histint',
       #'cloudcover_mid_histint', 'cloudcover_high_histint', 'windspeed_10mint',
       #'winddirection_10mint', 'shortwave_radiationint',
       #'direct_solar_radiation_histint', 'diffuse_radiationint',
       #'surface_solar_radiation_downwards_histint',
       #'10_metre_u_wind_component_histint',
       #'10_metre_v_wind_component_histint', 'diff_temperature',
       'diff_dewpoint', 'diff_cloudcover_total', 'diff_direct_solar_radiation',
       'diff_surface_solar_radiation_downwards', 'diff_snowfall',
       'diff_10_metre_u_wind_component', 'diff_10_metre_v_wind_component']
#drop unimportant features
weatherHistInt2 = weatherHistInt[histWfeat]

### add sunlight feature to pred weather
- TODO: we can add a relative sun power downwards normalized over minutes of max sunshine

In [6]:
from datetime import datetime
from suntime import Sun
import pytz

In [7]:
sunrise = []
sunset = []
for i in range(0,weatherPredInt.shape[0]):
    sun = Sun(weatherPredInt.latitude.iloc[i], weatherPredInt.longitude.iloc[i])
    dt = pd.to_datetime(weatherPredInt.forecast_datetime.iloc[i])
    sunrise.append(sun.get_sunrise_time(dt))
    sunset.append(sun.get_sunset_time(dt))

In [9]:
weatherPredInt['sunrise'] = sunrise
weatherPredInt['sunset'] = sunset

In [10]:
weatherPredInt['forecast_datetime'] = pd.to_datetime(weatherPredInt['forecast_datetime'])
def is_daylight(row):
    sunrise = row['sunrise']
    sunset = row['sunset']
    datetime = row['forecast_datetime']
    return sunrise <= datetime <= sunset
def calc_daylight(row):
    sunrise = row['sunrise']
    sunset = row['sunset']
    return (sunset-sunrise).total_seconds()/60
weatherPredInt['daylight'] = weatherPredInt.apply(is_daylight, axis=1)
weatherPredInt['minDaylight'] = weatherPredInt.apply(calc_daylight, axis=1)


### merge train and client

In [11]:
producing = train.loc[train.is_consumption == 0]
consuming = train.loc[train.is_consumption == 1]
train = pd.merge(producing.drop('is_consumption',axis = 1), consuming.drop('is_consumption',axis = 1),on=['data_block_id','prediction_unit_id','datetime','county','is_business','product_type'], how='outer',suffixes=('_prod', '_cons'))
del producing, consuming
print(train.shape)

(1009176, 10)


In [12]:
clientsTime = pd.merge(train, client, on=['county','is_business','product_type','data_block_id'], how='inner')
#clientsTime.dropna(inplace=True)

In [13]:
clientsTime['datetime'] = pd.to_datetime(clientsTime['datetime'])
clientsTime['yearday'] = clientsTime['datetime'].dt.day_of_year
clientsTime['weekday'] = clientsTime['datetime'].dt.day_of_week
clientsTime['month'] = clientsTime['datetime'].dt.month

In [14]:
unique_pairs = list(set(zip(clientsTime['is_business'], clientsTime[ 'product_type'])))
pair_index_dict = {pair: index for index, pair in enumerate(unique_pairs)}
clientsTime['business_prodType'] = list(map(pair_index_dict.get, zip(clientsTime['is_business'], clientsTime['product_type'])))

unique_pairs_cust = list(set(zip(clientsTime['is_business'], clientsTime[ 'product_type'], clientsTime['county'], clientsTime['eic_count'],clientsTime['installed_capacity'])))
pair_index_dict = {pair: index for index, pair in enumerate(unique_pairs_cust)}
clientsTime['ind_customer_id'] = list(map(pair_index_dict.get, zip(clientsTime['is_business'], clientsTime['product_type'], clientsTime['county'], clientsTime['eic_count'],clientsTime['installed_capacity'])))

In [17]:
(clientsTime.isna() == True).any()

county                False
is_business           False
product_type          False
target_prod            True
datetime              False
data_block_id         False
row_id_prod           False
prediction_unit_id    False
target_cons            True
row_id_cons           False
eic_count             False
installed_capacity    False
date                  False
yearday               False
weekday               False
month                 False
business_prodType     False
ind_customer_id       False
dtype: bool

In [19]:
#nan values when time change!!
clientsTime.loc[clientsTime.target_prod.isna()].datetime.value_counts()
#TODO: weather is in UCT, ePrices & clients not! -> need to deal with time difference!!
# nans for day light savings in ePrices & clients
# weatherInt is wrong!!!! the overlap to the time is different depending on the time difference!!!

datetime
2022-10-30 03:00:00    68
2022-03-27 03:00:00    67
2023-03-26 03:00:00    66
2021-10-31 03:00:00    63
Name: count, dtype: int64

In [27]:
weatherPredInt.loc[weatherPredInt.data_block_id == 208].forecast_datetime.value_counts()

forecast_datetime
2022-03-28 00:00:00+00:00    16
2022-03-28 01:00:00+00:00    16
2022-03-28 22:00:00+00:00    16
2022-03-28 21:00:00+00:00    16
2022-03-28 20:00:00+00:00    16
2022-03-28 19:00:00+00:00    16
2022-03-28 18:00:00+00:00    16
2022-03-28 17:00:00+00:00    16
2022-03-28 16:00:00+00:00    16
2022-03-28 15:00:00+00:00    16
2022-03-28 14:00:00+00:00    16
2022-03-28 13:00:00+00:00    16
2022-03-28 12:00:00+00:00    16
2022-03-28 11:00:00+00:00    16
2022-03-28 10:00:00+00:00    16
2022-03-28 09:00:00+00:00    16
2022-03-28 08:00:00+00:00    16
2022-03-28 07:00:00+00:00    16
2022-03-28 06:00:00+00:00    16
2022-03-28 05:00:00+00:00    16
2022-03-28 04:00:00+00:00    16
2022-03-28 03:00:00+00:00    16
2022-03-28 02:00:00+00:00    16
2022-03-28 23:00:00+00:00    16
Name: count, dtype: int64

# prepare data into trainable arrays
first axis should be data block id

In [22]:
#clientsTime, weatherPredInt, weatherHistInt, ePrices, gasPrices

featPredWeather = [
        #'latitude', 'longitude', 
        'County', #'forecast_datetime',
       'hours_ahead',
        'data_block_id', #'origin_datetime', 
       'temperatureint',
       'dewpointint', 'cloudcover_highint', 'cloudcover_lowint',
       'cloudcover_midint', 'cloudcover_totalint',
       '10_metre_u_wind_componentint', '10_metre_v_wind_componentint',
       'direct_solar_radiationint', 'surface_solar_radiation_downwardsint',
       'snowfallint', 'total_precipitationint', 
       #'sunrise', 'sunset',
       'daylight', 'minDaylight']

predWeatherComplete = weatherPredInt[featPredWeather]

featHistWeather = [
        'County', #'forecast_datetime',
       'hours_ahead', 'data_block_id',# 'origin_datetime',
       #'temperature_predint', 'dewpoint_predint', 'cloudcover_high_predint',
       #'cloudcover_low_predint', 'cloudcover_mid_predint',
       #'cloudcover_total_predint', '10_metre_u_wind_componentint',
       #'10_metre_v_wind_componentint', 'direct_solar_radiation_predint',
       #'surface_solar_radiation_downwardsint', 'snowfall_predint',
       #'total_precipitationint', 'temperature_histint', 'dewpoint_histint',
       #'rainint', 'snowfall_histint', 'surface_pressureint',
       #'cloudcover_total_histint', 'cloudcover_low_histint',
       #'cloudcover_mid_histint', 'cloudcover_high_histint', 'windspeed_10mint',
       #'winddirection_10mint', 'shortwave_radiationint',
       #'direct_solar_radiation_histint', 'diffuse_radiationint',
       #'surface_solar_radiation_downwards_histint',
       #'10_metre_u_wind_component_histint',
       #'10_metre_v_wind_component_histint', 'diff_temperature',
       'diff_dewpoint', 'diff_cloudcover_total', 'diff_direct_solar_radiation',
       'diff_surface_solar_radiation_downwards', 'diff_snowfall',
       'diff_10_metre_u_wind_component', 'diff_10_metre_v_wind_component'
]
histWeatherComplete = weatherHistInt[featHistWeather]

# can't use client id as an axis in data because it's not constant!
y = np.zeros((clientsTime.shape[0],2))
y_indexes = np.zeros((clientsTime.shape[0],2)) # dataframe indexes in the end, not important for now
index_y = 0

customers = []

constValsArray = np.zeros((1,8))
targetsArray = np.zeros((1,24,2))
ePricesArray = np.zeros((1,24,1))

nFeatHistWeather = histWeatherComplete.shape[1]
histWeatherArray = np.zeros((1,10,nFeatHistWeather))
nFeatPredWeather = predWeatherComplete.shape[1]
predWeatherArray = np.zeros((1,24,nFeatPredWeather))
dataBlockIdArray = np.zeros((1))

# loop over customer, append all data cycles for each customer
for customerId in clientsTime.ind_customer_id.unique():
    customerSlice = clientsTime.loc[clientsTime.ind_customer_id == customerId]

    # const values
    county       = customerSlice.county.unique()[0]
    is_business  = customerSlice.is_business.unique()[0]
    product_type = customerSlice.product_type.unique()[0]
    prediction_unit_id = customerSlice.prediction_unit_id.unique()[0] #should be redundant
    eic_count          = customerSlice.eic_count.unique()[0]
    installed_capacity = customerSlice.installed_capacity.unique()[0]
    yearday = customerSlice.yearday.unique()[0]
    weekday = customerSlice.weekday.unique()[0]
    month   = customerSlice.month.unique()[0]
    business_prodType = customerSlice.business_prodType.unique()[0]
    ind_customer_id   = customerSlice.ind_customer_id.unique()[0]

    customer = {}
    customer['contVals'] = [county, is_business, product_type, prediction_unit_id, eic_count, installed_capacity, business_prodType, ind_customer_id]
    
    for dataBlockId in customerSlice.data_block_id.unique():
        timeSlice = customerSlice.loc[customerSlice.data_block_id == dataBlockId]
        gasSlice = gasPrices.loc[gasPrices.data_block_id == dataBlockId]
        eSlice   = ePrices.loc[ePrices.data_block_id == dataBlockId]

        lowest_price_per_mwh = gasSlice.lowest_price_per_mwh.iloc[0]
        highest_price_per_mwh = gasSlice.highest_price_per_mwh.iloc[0]
        euros_per_mwh = eSlice['euros_per_mwh'].to_numpy()
        if euros_per_mwh.shape[0] == 23:
            print(dataBlockId, customerId)

        y_cons = timeSlice['target_cons'] / installed_capacity
        y_prod = timeSlice['target_prod'] / installed_capacity
        #y[index_y:index_y+24,0] = y_cons
        #y[index_y:index_y+24,1] = y_prod

        #y_indexes[index_y:index_y+24,0] = timeSlice['row_id_cons']
        #y_indexes[index_y:index_y+24,1] = timeSlice['row_id_prod']

        histWeather = histWeatherComplete.loc[(histWeatherComplete.data_block_id == dataBlockId) & (histWeatherComplete.County == county)]
        predWeather = predWeatherComplete.loc[(predWeatherComplete.data_block_id == dataBlockId) & (predWeatherComplete.County == county)]
        #predWeather['ePrices'] = euros_per_mwh
        if y_cons.shape[0] == 23:
            print(dataBlockId, customerId)
        customerDay = {}
        customerDay['y_cons'] = y_cons
        customerDay['y_prod'] = y_prod
        customerDay['ePrices'] = euros_per_mwh
        customerDay['timeBlockConst'] = [lowest_price_per_mwh, highest_price_per_mwh,yearday, weekday, month]
        customerDay['histWeather'] = histWeather
        customerDay['predWeather'] = predWeather
        customer['data_block'] = customerDay

        new_row = np.array([county, is_business, product_type, prediction_unit_id, eic_count, installed_capacity, business_prodType, ind_customer_id])
        constValsArray = np.vstack((constValsArray, new_row))

        new_row = np.zeros((1,24,2))
        new_row[:,:,0] = y_cons
        new_row[:,:,1] = y_prod
        targetsArray = np.concatenate((targetsArray, new_row), axis=0)

        new_row = np.zeros((1,24,1))
        new_row[0,:,0] = euros_per_mwh
        ePricesArray = np.concatenate((ePricesArray, new_row), axis=0)

        new_row = np.zeros((1,10,nFeatHistWeather))
        new_row[:,:,:] = histWeather
        histWeatherArray = np.concatenate((histWeatherArray, new_row), axis=0)

        new_row = np.zeros((1,24,nFeatPredWeather))
        new_row[:,:,:] = predWeather
        predWeatherArray = np.concatenate((predWeatherArray, new_row), axis=0)

        dataBlockIdArray = np.concatenate((dataBlockIdArray, np.array([dataBlockId])))
    customers.append(customer)

    


208 2088


ValueError: could not broadcast input array from shape (23,) into shape (24,)

In [168]:
clientsTime.loc[(clientsTime.ind_customer_id == 394) & (clientsTime.data_block_id == 60)]

,county,is_business,product_type,target_prod,datetime,data_block_id,row_id_prod,prediction_unit_id,target_cons,row_id_cons,eic_count,installed_capacity,date,yearday,weekday,month,business_prodType,ind_customer_id
89448,1,0,1,0.000,2021-10-31 00:00:00,60,178574.0,6.0,6.573,178575.0,7.0,80.0,2021-10-29,304,6,10,0,394
89449,1,0,1,0.000,2021-10-31 01:00:00,60,178700.0,6.0,7.235,178701.0,7.0,80.0,2021-10-29,304,6,10,0,394
89450,1,0,1,0.000,2021-10-31 02:00:00,60,178826.0,6.0,5.207,178827.0,7.0,80.0,2021-10-29,304,6,10,0,394
89452,1,0,1,0.000,2021-10-31 04:00:00,60,179078.0,6.0,5.636,179079.0,7.0,80.0,2021-10-29,304,6,10,0,394
89453,1,0,1,0.000,2021-10-31 05:00:00,60,179204.0,6.0,5.561,179205.0,7.0,80.0,2021-10-29,304,6,10,0,394
89454,1,0,1,0.000,2021-10-31 06:00:00,60,179330.0,6.0,6.479,179331.0,7.0,80.0,2021-10-29,304,6,10,0,394
89455,1,0,1,0.060,2021-10-31 07:00:00,60,179456.0,6.0,6.064,179457.0,7.0,80.0,2021-10-29,304,6,10,0,394
89456,1,0,1,2.589,2021-10-31 08:00:00,60,179582.0,6.0,6.680,179583.0,7.0,80.0,2021-10-29,304,6,10,0,394
89457,1,0,1,5.561,2021-10-31 09:00:00,60,179708.0,6.0,5.160,179709.0,7.0,80.0,2021-10-29,304,6,10,0,394
89458,1,0,1,16.843,2021-10-31 10:00:00,60,179834.0,6.0,3.691,179835.0,7.0,80.0,2021-10-29,304,6,10,0,394


In [ ]:
#remove first rows
targetsArray = targetsArray[1:-2,:,:]

In [161]:
histWeatherArray

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.0

# model

In [ ]:
# Define the inputs
time_inputs = tf.keras.Input(shape=(24, 30), name='time_inputs')
additional_input = tf.keras.Input(shape=(10, 15), name='additional_input')
constant_inputs = tf.keras.Input(shape=(6,), name='constant_inputs')

# Process the time-based inputs
time_flattened = layers.Flatten()(time_inputs)
time_dense = layers.Dense(64, activation='relu')(time_flattened)

# Concatenate all inputs
merged_inputs = layers.Concatenate()([time_dense, additional_input, constant_inputs])

# Main dense block
x = layers.Dense(128, activation='relu')(merged_inputs)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Output layer for 24*2 targets
output_layer = layers.Dense(24 * 2, activation='linear', name='output')(x)

# Define the model
model = models.Model(inputs=[time_inputs, additional_input, constant_inputs], outputs=output_layer)

# Compile the model with an appropriate loss function and optimizer
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Display the model summary
model.summary()
